In [1]:
import numpy as np
np.random.seed(42)
import matplotlib.pyplot as pl
from tqdm import tqdm
import celeriteflow as cf
import keras
keras.backend.set_floatx("float64")

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def GetTestData(clobber=False):
    """Get test data."""
    data = np.load("data/c6/test_data.npz")
    reg_fluxes = data['reg_fluxes']
    reg_pixels = data['reg_pixels']
    reg_npix = data['reg_npix']
    time = data['time']
    flux = data['flux']
    flux_err = data['flux_err']
    return time, reg_fluxes, reg_pixels, reg_npix, flux, flux_err

In [ ]:
# Grab the data
time, reg_fluxes, reg_pixels, reg_npix, flux, flux_err = GetTestData()
ntime = flux.shape[0]
nreg = reg_fluxes.shape[0]

# Set up tensorflow
import tensorflow as tf
dtype = tf.float64

# Tweakable stuff
logjitter0 = np.log(0.01)  # Initial log jitter
l0 = 1e-2  # Initial L2 regularization variance
learning_rate = 1e-2  # Initial Adam learning rate
niter = 500  # Number of iterations
P0 = 10.0  # Period guess
H = 2  # Dimension of the hidden layer

# This is our data we want to fit
y = tf.constant(flux - 1.0, dtype=dtype)

# This is the data uncertainty
y_err = tf.constant(flux_err, dtype=dtype)

# Compute weakly regularized max like solution as a guess
X = reg_fluxes.T - 1.0
L = np.diag(np.ones(nreg) * l0 ** 2)
LXT = np.dot(L, X.T)
S = np.dot(X, np.dot(L, X.T))
S += np.diag(flux_err ** 2 + np.exp(2 * logjitter0))
Sinvy = np.linalg.solve(S, (flux - 1.0)[:, None])
wguess = np.dot(LXT, Sinvy)

# Keras NN model
nn = keras.Sequential()
nn.add(keras.layers.Dense(H, activation="softmax", input_dim=nreg))
nn.add(keras.layers.Dense(1))
feed_dict = {nn.input: reg_fluxes.T - 1.0}
model = tf.squeeze(nn.output)

# Celerite GP
logjitter = tf.Variable(logjitter0, dtype=dtype)
t = tf.constant(time, dtype=dtype)
diag = y_err ** 2 + tf.exp(2*logjitter)
resid = y - model
log_S0 = tf.Variable(np.log(np.var(flux)), dtype=dtype)
log_w0 = tf.Variable(np.log(2 * np.pi / P0), dtype=dtype)
log_Q = tf.Variable(0.0, dtype=dtype)
kernel = cf.terms.SHOTerm(log_S0=log_S0,
                          log_w0=log_w0,
                          log_Q=log_Q)
gp = cf.GaussianProcess(kernel, t, resid, diag)
loglike = gp.log_likelihood

# Losses
l = tf.constant(l0, dtype=dtype)
loss = -2 * loglike
for w in nn.trainable_weights:
    loss += (1 / l) * tf.reduce_sum(tf.abs(w))
opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# Init session
session = tf.get_default_session()
if session is None:
    session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())

# Assign initial guess for the weights
w1_0 = 1e-6 * np.random.randn(nreg, H)
w1_0[:, 0] = wguess[:, 0]
w2_0 = 1e-6 * np.random.randn(H, 1)
w2_0[0, 0] = 1
session.run(tf.assign(nn.layers[0].kernel, w1_0))
session.run(tf.assign(nn.layers[0].bias, 0 * np.random.randn(H)))
session.run(tf.assign(nn.layers[1].kernel, w2_0))
session.run(tf.assign(nn.layers[1].bias, [-0.5]))

# Plot initial model
model0 = model.eval(feed_dict=feed_dict)
fig, ax = pl.subplots(2)
ax[0].set_title("Final model")
ax[0].plot(time, flux - 1, 'k.', ms=2, alpha=0.3)
ax[0].plot(time, model0, 'r-', lw=0.5)

# Iterate!
losses = np.zeros(niter)
best_loss = np.inf
for i in tqdm(range(niter)):
    session.run(opt, feed_dict=feed_dict)
    losses[i] = loss.eval(feed_dict=feed_dict)
    if losses[i] < best_loss:
        best_loss = losses[i]
        best_weights = session.run(nn.trainable_weights)
        best_logjitter = logjitter.eval()
        best_log_S0 = log_S0.eval()
        best_log_w0 = log_w0.eval()
        best_log_Q = log_Q.eval()
session.run([
    tf.assign(a, b) for a, b in zip(nn.trainable_weights, best_weights)])
session.run(tf.assign(logjitter, best_logjitter))
session.run(tf.assign(log_S0, best_log_S0))
session.run(tf.assign(log_w0, best_log_w0))
session.run(tf.assign(log_Q, best_log_Q))

# Log
print("GP stuff:", best_logjitter, best_log_S0, best_log_w0, best_log_Q)

# Plot learning rate
fig, ax = pl.subplots(1)
ax.plot(range(niter), losses)

# Plot weights
'''
for w in best_weights:
    fig, ax = pl.subplots(1)
    ax.imshow(np.log10(np.abs(w)), aspect='auto')
'''

# Plot initial model
'''
fig, ax = pl.subplots(2)
ax[0].set_title("Initial model")
ax[0].plot(time, flux, 'k.', ms=2, alpha=0.3)
ax[0].plot(time, 1 + model0, 'r-', lw=0.5)
ax[1].plot(time, flux - model0, 'k.', ms=2, alpha=0.3)
'''

# Plot final model
fig, ax = pl.subplots(2)
ax[0].set_title("Final model")
ax[0].plot(time, flux, 'k.', ms=2, alpha=0.3)
ax[0].plot(time, 1 + model.eval(feed_dict=feed_dict), 'r-', lw=0.5)
ax[1].plot(time, flux - model.eval(feed_dict=feed_dict), 'k.', ms=2, alpha=0.3)

  6%|▌         | 29/500 [00:19<05:22,  1.46it/s]